In [36]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [37]:
!pip install gdown

In [38]:
df_rds = pd.read_csv("data/Validations/validations_rds.csv",sep = ";")
df_rds

,JOUR,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_LIGNE,LIBELLE_LIGNE,ID_GROUPOFLIGNE,CATEGORIE_TITRE,NB_VALD
0,2025-05-12,100,100,366,366 (Colomb'bus),A01165,Contrat Solidarité Transport,427
1,2025-05-12,100,100,366,366 (Colomb'bus),A01165,Forfaits courts,68
2,2025-05-12,100,100,367,367,A01166,Amethyste,161
3,2025-05-12,100,100,367,367,A01166,Forfait Navigo,1742
4,2025-05-12,100,100,368,368,A01167,Contrat Solidarité Transport,442
...,...,...,...,...,...,...,...,...
713901,2025-06-29,100,987,783,N14,A01269,Contrat Solidarité Transport,251
713902,2025-06-29,100,987,783,N14,A01269,NON DEFINI,6
713903,2025-06-29,100,987,784,N13,A01270,Imagine R,219
713904,2025-06-29,100,987,784,N13,A01270,NON DEFINI,8


In [39]:
df_t2 = df_rds[df_rds['ID_GROUPOFLIGNE']== "A01192"] #équivalent du WHERE, on sélectionne le T2 avec son ID unique
val_jour_t2 = df_t2.groupby("JOUR")["NB_VALD"].sum() #nb de validations sur chaque jour
val_jour_t2

#ça l'a converti en Series

JOUR
2025-04-01    117635
2025-04-02    117218
2025-04-03    119297
2025-04-04    108250
2025-04-05     70775
               ...  
2025-06-26    118192
2025-06-27    112052
2025-06-28     69663
2025-06-29     50427
2025-06-30    115129
Name: NB_VALD, Length: 91, dtype: int64

In [40]:
df_62 = df_rds[df_rds["LIBELLE_LIGNE"] == "62"] #on retrouve le 62

In [41]:
chq_ligne = (
    df_rds.groupby(['ID_GROUPOFLIGNE', 'LIBELLE_LIGNE'], as_index=False)['NB_VALD']
    .sum()
    .sort_values(by='NB_VALD', ascending=False)
)
chq_ligne['NB_VALD_MOY_JOUR'] = (chq_ligne['NB_VALD']/91).round(0) #pour arrondir : round(0)
chq_ligne.head(30)

#chq_ligne[chq_ligne["LIBELLE_LIGNE"].astype(str).str.contains("25")]
#chq_ligne[chq_ligne["LIBELLE_LIGNE"] == "96"]

,ID_GROUPOFLIGNE,LIBELLE_LIGNE,NB_VALD,NB_VALD_MOY_JOUR
981,A01193,T3a,10700314,117586.0
1228,A01757,T3b,10662056,117165.0
980,A01192,T2,8621139,94738.0
0,A00000,LIGNE NON DEFINIE,6242241,68596.0
1414,A02371,T9 - Paris Porte de Choisy - Orly Gaston Viens,5622812,61789.0
736,A00924,TVM,4912649,53985.0
1268,A02149,T6,4029208,44277.0
763,A00952,62,2776453,30510.0
1269,A02150,T8,2488586,27347.0
1231,A01760,T5,2460480,27038.0


In [42]:
#profil horaire, ça nous servira plus tard
df_ph = pd.read_csv("data/Validations/profil_horaire_rds.csv",sep = ";")
df_ph.head(10)

,CODE_STIF_TRNS,CODE_STIF_RES,CODE_STIF_LIGNE,LIBELLE_LIGNE,ID_GROUPOFLIGNE,CAT_JOUR,TRNC_HORR_60,Pourcentage_validations
0,513,513,016,LIEUSAINT - SERRIS,A00025,SAVS,10H-11H,1.39
1,513,513,016,LIEUSAINT - SERRIS,A00025,SAVS,12H-13H,4.88
2,513,513,016,LIEUSAINT - SERRIS,A00025,SAVS,15H-16H,3.83
3,513,513,016,LIEUSAINT - SERRIS,A00025,SAVS,9H-10H,1.05
4,513,513,018,TOURNAN-EN-BRIE - TORCY (RER),A00015,DIJFP,18H-19H,18.64
5,513,513,018,TOURNAN-EN-BRIE - TORCY (RER),A00015,JOHV,8H-9H,20.11
6,513,513,018,TOURNAN-EN-BRIE - TORCY (RER),A00015,JOVS,16H-17H,16.91
7,513,513,002,ligne régulière 2,A02771,DIJFP,13H-14H,11.90
8,513,513,002,ligne régulière 2,A02771,DIJFP,16H-17H,10.54
9,513,513,002,ligne régulière 2,A02771,DIJFP,19H-20H,7.23


In [43]:
#calcul de l'offre. Dans le GTFS il y a aussi le réseau ferré.
agency = pd.read_csv("data/GTFS/agency.txt") #associe agency_id avec réseau de transport. Go trouver Paris-Saclay
routes = pd.read_csv("data/GTFS/routes.txt") #associe route_id avec une ligne de transport
trips = pd.read_csv("data/GTFS/trips.txt") #lien avec routes par route_id / calendar par service_id / stop_time par trip_id
calendar = pd.read_csv("data/GTFS/calendar.txt") #quel jour il y a tel service, identifié par service_id
calendar_dates = pd.read_csv("data/GTFS/calendar_dates.txt") #services à ajouter/retirer car exceptions
stop_time = pd.read_csv("data/GTFS/stop_times.txt") #heure d'arrivée de trip_id à stop_id
stops = pd.read_csv("data/GTFS/stops.txt") #associe stop_id avec un arrêt physique

""" 
!!! Lien entre GTFS et données de validation : route_id = IDFM:ID_ligne
 
Les arrêts aussi peuvent être identifiés facilement comme ça si tu as envie
"""

"""
merge : à gauche c'est ta base, ce que tu gardes. A droite, c'est ce que tu ajoutes
"""


/tmp/ipykernel_8281/4128150223.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  trips = pd.read_csv("data/GTFS/trips.txt") #lien avec routes par route_id / calendar par service_id / stop_time par trip_id
/tmp/ipykernel_8281/4128150223.py:7: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  stop_time = pd.read_csv("data/GTFS/stop_times.txt") #heure d'arrivée de trip_id à stop_id


"\nmerge : à gauche c'est ta base, ce que tu gardes. A droite, c'est ce que tu ajoutes\n"

In [44]:
#Plein de jointures

agency_routes = routes.merge(agency,left_on = "agency_id",right_on = "agency_id",how="left")

stop_time_stop = stops.merge(stop_time,left_on="stop_id",right_on="stop_id",how="left")

saclay_gtfs = agency_routes[agency_routes["agency_id"] == "IDFM:1060"] #agency_id : IDFM:1060 pour réseau Paris-Saclay
saclay_gtfs = saclay_gtfs[["agency_id","route_id","route_long_name"]]

trips_saclay = saclay_gtfs.merge(trips,left_on="route_id",right_on="route_id",how="left")
trips_saclay = trips_saclay[["route_id","trip_id","service_id","direction_id","route_long_name"]]

calendar_saclay = trips_saclay.merge(calendar,left_on="service_id",right_on="service_id",how="left")

horaires_saclay = calendar_saclay.merge(stop_time_stop,left_on="trip_id",right_on="trip_id",how="left")
horaires_saclay = horaires_saclay[["route_long_name","trip_id","stop_name","arrival_time","departure_time","monday"]]
horaires_4606 = horaires_saclay[horaires_saclay["route_long_name"] == "4606 (ex 91.06)"]


In [45]:
horaires_4606_lundi = horaires_4606[horaires_4606["monday"] == 1]
horaires_4606_lundi["trip_id"].nunique()

281

In [46]:
#compter combien de trips : un trip = une course, donc Terminus1 -> Terminus2
#Ici on a compté le nombre de trips "le lundi", et pas sur un lundi représentatif. Donc on surestime probablement l'offre.
horaires_saclay_lundi = horaires_saclay[horaires_saclay["monday"]==1]

trip_par_ligne_lundi = (horaires_saclay_lundi.groupby(
    ["route_long_name"],as_index = False)["trip_id"].nunique()
)
trip_par_ligne_lundi

,route_long_name,trip_id
0,4601 (ex 1),77
1,4602 (ex 2),239
2,4603 (ex 3),186
3,4604 (ex 4),95
4,4605 (ex 5),113
5,4606 (ex 91.06),281
6,4607 (ex 7),134
7,4608 (ex 8),56
8,4609 (ex 9),222
9,4610 (ex 10),38


In [47]:
#Offre sur un jour

services_jour = (calendar.groupby(
    ["start_date"],as_index=False)["service_id"].nunique()
) #nombre de services qui commencent tel jour
#On prend le lundi 29 septembre, il a l'air bien au milieu de tout

date = pd.Timestamp("2025-09-29")
dint = int(date.strftime("%Y%m%d")) #conversion au même format que les dates des données GTFS

#Services actifs le 29/09
actifs = calendar[
    (calendar["start_date"] <= dint) &
    (calendar["end_date"]   >= dint) &
    (calendar["monday"] == 1)
]["service_id"]

#Exceptions du 29/09
exceptions = calendar_dates[calendar_dates["date"] == dint]
actifs = pd.Index(actifs).unique()

enlever = pd.Index(exceptions.loc[exceptions["exception_type"] == 2, "service_id"]).unique()
ajouter = pd.Index(exceptions.loc[exceptions["exception_type"] == 1, "service_id"]).unique()
#loc : sert à sélectionner certaines lignes et certaines colonnes, avec potentiellement des booléens

actifs = actifs.difference(enlever)
actifs = actifs.union(ajouter)
d_actifs = {}
for service_id in actifs :
    d_actifs[service_id] = 1

In [48]:
trips_2909 = trips.loc[trips["service_id"].isin(actifs)]
#isin : vérifie pour chaque élément du DataFrame s'il est dans actifs

offre_2909 = trips_2909.merge(routes,on="route_id",how="left")
offre_2909 = offre_2909[["route_id","route_long_name","trip_id","direction_id"]]
offre_2909

,route_id,route_long_name,trip_id,direction_id
0,IDFM:C02060,Filéo Roissy Sud - Villeparisis et Mitry (sur ...,IDFM:KRO6:208019-C02060-696-260,1
1,IDFM:C01792,N2,IDFM:stif:local-175995-C01792-5828,0
2,IDFM:C00562,N1,IDFM:stif:local-175995-C00562-5439,1
3,IDFM:C01792,N2,IDFM:stif:local-175995-C01792-5788,0
4,IDFM:C00563,CDG VAL,IDFM:stif:local-175995-C00563-5995,0
...,...,...,...,...
142882,IDFM:C01731,R,IDFM:TN:SNCF:1f49e5ab-e3e2-4365-9acc-aa0342ed6a39,1
142883,IDFM:C01731,R,IDFM:TN:SNCF:3cb15ed5-0c96-4b3e-b128-5659386dbb56,1
142884,IDFM:C01841,P,IDFM:TN:SNCF:b8026a77-21eb-42de-a023-71097be653be,1
142885,IDFM:C01728,D,IDFM:TN:SNCF:2e832998-dc52-4711-afa7-49d50841bfdd,1


In [49]:
#Heure (ou minute) de départ du trip au premier arrêt
st = stop_time.loc[stop_time["trip_id"].isin(offre_2909["trip_id"])].copy()
st = st.sort_values(["trip_id", "stop_sequence"])
first_dep = st.groupby("trip_id", as_index=False).first()[["trip_id", "departure_time"]]

#Conversion de HH:MM:SS en minutes depuis 00:00 (gère par ex "25:10:00" si nécessaire)
def to_min(t):
    #marche même si t dépasse 24:00:00
    h, m, s = map(int, t.split(":"))
    return h*60 + m + s/60

first_dep["dep_min"] = first_dep["departure_time"].map(to_min)

dep = first_dep.merge(offre_2909[["trip_id","route_id","direction_id"]], on="trip_id", how="left")

#Pointe du matin 7h–10h
peak = dep[(dep["dep_min"] >= 7*60) & (dep["dep_min"] < 10*60)]

freq = (peak.groupby(["route_id","direction_id"], as_index=False)
             .size()
             .rename(columns={"size": "voyages_3h"}))
freq["voyages_h"] = freq["voyages_3h"] / 3.0
freq["headway_min"] = 60.0 / freq["voyages_h"]

freq = freq.merge(
    routes[["route_id","route_short_name","route_long_name"]],
    on="route_id", how="left"
)

In [50]:
freq_triée = freq.sort_values(by="voyages_h",ascending=False)
"""
with pd.option_context(
    "display.max_rows", 1000,
    "display.max_columns", None,
    "display.width", None,
    "display.max_colwidth", None
):
    display(freq_triée.head(200))"""

'\nwith pd.option_context(\n    "display.max_rows", 1000,\n    "display.max_columns", None,\n    "display.width", None,\n    "display.max_colwidth", None\n):\n    display(freq_triée.head(200))'

In [51]:
freq_routes = freq.merge(routes,on="route_id",how="left")
freq_saclay = freq_routes[freq_routes["agency_id"] == "IDFM:1060"]
freq_4606 = freq_saclay[freq_saclay["route_long_name_x"] == "4606 (ex 91.06)"]

freq_4606[["route_long_name_x","direction_id","voyages_h","headway_min"]] 
#on trouve un bus tous les 4 min en pointe sur le 4606, c'est cohérent !!

,route_long_name_x,direction_id,voyages_h,headway_min
1879,4606 (ex 91.06),0,14.333333,4.186047
1880,4606 (ex 91.06),1,15.000000,4.000000


In [52]:
#Données du réseau ferré

df_rf = pd.read_csv("data/Validations/validations_rf.csv",sep=";")

chq_arret = (
    df_rf.groupby(['ID_ZDC', 'LIBELLE_ARRET'], as_index=False)['NB_VALD']
    .sum()
    .sort_values(by='NB_VALD', ascending=False)
)
chq_arret['NB_VALD_MOY_JOUR'] = (chq_arret['NB_VALD']/91).round(0) #pour arrondir : round(0)
chq_arret.head(10)

,ID_ZDC,LIBELLE_ARRET,NB_VALD,NB_VALD_MOY_JOUR
482,71370.0,SAINT-LAZARE,20559184.0,225925.0
440,71264.0,CHATELET,13665229.0,150167.0
678,73626.0,GARE DE LYON,12328284.0,135476.0
514,71517.0,LA DEFENSE,10767994.0,118330.0
401,71139.0,MONTPARNASSE,10255590.0,112699.0
491,71410.0,GARE DU NORD,9182447.0,100906.0
478,71359.0,GARE DE L'EST,7987855.0,87779.0
715,73794.0,LES HALLES,4444820.0,48844.0
458,71311.0,REPUBLIQUE,4296079.0,47210.0
473,71347.0,CH.D.G.ETOILE,4217050.0,46341.0


In [53]:
df_ph_rf = pd.read_csv("data/Validations/profil_horaire_rf.csv",sep=";")
merge_rf = chq_arret.merge(df_ph_rf,left_on = "ID_ZDC",right_on = "ID_ZDC",how="left")
merge_rf["VALD_HEURE"] = merge_rf["NB_VALD_MOY_JOUR"]*0.01*merge_rf["Pourcentage_validations"]
merge_rf = merge_rf[["LIBELLE_ARRET_x","NB_VALD_MOY_JOUR","TRNC_HORR_60","Pourcentage_validations","VALD_HEURE"]]
merge_rf = merge_rf.sort_values(by="VALD_HEURE",ascending=False)
merge_rf

,LIBELLE_ARRET_x,NB_VALD_MOY_JOUR,TRNC_HORR_60,Pourcentage_validations,VALD_HEURE
222,SAINT-LAZARE,225925.0,18H-19H,16.58,37458.3650
82,SAINT-LAZARE,225925.0,18H-19H,16.34,36916.1450
61,SAINT-LAZARE,225925.0,17H-18H,14.30,32307.2750
177,SAINT-LAZARE,225925.0,8H-9H,14.09,31832.8325
42,SAINT-LAZARE,225925.0,17H-18H,14.01,31652.0925
...,...,...,...,...,...
14244,MAISON BLANCHE,15411.0,4H-5H,0.00,0.0000
61975,GENNEVILLIERS,2812.0,0H-1H,0.00,0.0000
3815,PLACE D'ITALIE,27564.0,3H-4H,0.00,0.0000
60550,ACHERES VILLE,3086.0,3H-4H,0.00,0.0000


In [54]:
#on push tous les fichiers de données sauf stop_times.txt pour l'instant

Partie Open street map : 
On va relier les stations IDFM à leur environnement urbain (POI OSM).
L'idée est d'utiliser stops.txt, sélectionner un sous-échantillon (au début, pour tester sans se faire bloquer par OSM), définir les catégories de POI à récupérer (boulangeries, écoles, restaurants, etc.), faire une boucle sur les stations, appeler ox.features_from_point(), compter les POI par catégorie dans deux rayons (400 m et 800 m), stocker les résultats dans une listeet créer un DataFrame final et sauvegarder en CSV

In [55]:
!pip install geopandas osmnx requests shapely folium

In [56]:
import os, gdown

#lien : https://drive.google.com/file/d/1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS/view?usp=sharing
file_id = "1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS"
url = f"https://drive.google.com/uc?id={file_id}"
os.makedirs("data", exist_ok=True)
out_path = "data/GTFS/stop_times.txt"
gdown.download(url, out_path, quiet=False)  # gère aussi les gros fichiers >100 Mo

st = pd.read_csv("data/GTFS/stop_times.txt")
st.head(10)

Downloading...
From (original): https://drive.google.com/uc?id=1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS
From (redirected): https://drive.google.com/uc?id=1qLf7bU6Z2rVIvQ2y0dZYmiA9CGluwCMS&confirm=t&uuid=501d6613-b717-4ac7-bf3e-7904f8001d87
To: /home/onyxia/work/Projet-data-science/data/GTFS/stop_times.txt
100%|██████████| 883M/883M [00:20<00:00, 44.0MB/s] 
/tmp/ipykernel_8281/3665101781.py:10: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  st = pd.read_csv("data/GTFS/stop_times.txt")


,trip_id,arrival_time,departure_time,start_pickup_drop_off_window,end_pickup_drop_off_window,stop_id,stop_sequence,pickup_type,drop_off_type,local_zone_id,stop_headsign,timepoint,pickup_booking_rule_id,drop_off_booking_rule_id
0,IDFM:KRO6:208019-C02060-696-260,24:39:00,24:39:00,NaN,NaN,IDFM:477725,0,2,1,NaN,NaN,0,IDFM:12,IDFM:12
1,IDFM:KRO6:208019-C02060-696-260,24:40:00,24:40:00,NaN,NaN,IDFM:38175,1,2,2,NaN,NaN,0,IDFM:12,IDFM:12
2,IDFM:KRO6:208019-C02060-696-260,24:41:00,24:41:00,NaN,NaN,IDFM:41650,2,2,2,NaN,NaN,0,IDFM:12,IDFM:12
3,IDFM:KRO6:208019-C02060-696-260,24:42:00,24:42:00,NaN,NaN,IDFM:490575,3,2,2,NaN,NaN,0,IDFM:12,IDFM:12
4,IDFM:KRO6:208019-C02060-696-260,24:42:00,24:42:00,NaN,NaN,IDFM:38178,4,2,2,NaN,NaN,0,IDFM:12,IDFM:12
5,IDFM:KRO6:208019-C02060-696-260,24:43:00,24:43:00,NaN,NaN,IDFM:38179,5,2,2,NaN,NaN,0,IDFM:12,IDFM:12
6,IDFM:KRO6:208019-C02060-696-260,24:43:00,24:43:00,NaN,NaN,IDFM:38166,6,2,2,NaN,NaN,0,IDFM:12,IDFM:12
7,IDFM:KRO6:208019-C02060-696-260,24:43:00,24:43:00,NaN,NaN,IDFM:490576,7,2,2,NaN,NaN,0,IDFM:12,IDFM:12
8,IDFM:KRO6:208019-C02060-696-260,24:44:00,24:44:00,NaN,NaN,IDFM:490580,8,2,2,NaN,NaN,0,IDFM:12,IDFM:12
9,IDFM:KRO6:208019-C02060-696-260,24:44:00,24:44:00,NaN,NaN,IDFM:38167,9,2,2,NaN,NaN,0,IDFM:12,IDFM:12


In [57]:
#Open street map 
#récupérer les points d’intérêt (écoles, commerces, hôpitaux, musées, stades, etc.) dans un rayon autour des stations.
import osmnx as ox 
#permet d’accéder directement aux données d’OpenStreetMap (OSM) : rues, bâtiments, POI (points d’intérêt), etc.
#Elle interroge automatiquement l’API Overpass, un service qui renvoie des données géographiques OSM au format GeoJSON.

tags = {'amenity': True, 'shop': True, 'tourism': True, 'leisure': True}
#Exemple des POI trouvés dans un rayon de 500m autour de Notre Dame de Paris
poi = ox.features_from_point((48.8566, 2.3522), dist=500, tags=tags)
poi.head()


geometry   historic   latitude longitude  \
element id                                                                    
node    191031796    POINT (2.3488 48.8534)  milestone  48.853402  2.348785   
        218117881  POINT (2.34971 48.85695)        NaN        NaN       NaN   
        218119662   POINT (2.35352 48.8563)        NaN        NaN       NaN   
        218145208  POINT (2.35887 48.85697)        NaN        NaN       NaN   
        243461578  POINT (2.34644 48.85533)        NaN        NaN       NaN   

                       man_made                             name  \
element id                                                         
node    191031796  survey_point  Point zéro des Routes de France   
        218117881  surveillance           Parking Hôtel de Ville   
        218119662           NaN                     Lobau Rivoli   
        218145208           NaN            Synagogue Beit Yossef   
        243461578           NaN                           Lutèce   

                                                             name:el  \
element id                                                             
node    191031796  Σημείο μηδέν για την μέτρηση αποστάσεων στην Γ...   
        218117881                                                NaN   
        218119662                                                NaN   
        218145208                                                NaN   
        243461578                                                NaN   

                                                   name:en  \
element id                                                   
node    191031796  Point Zero for French Road Measurements   
        218117881                                      NaN   
        218119662                                      NaN   
        218145208                                      NaN   
        243461578                                      NaN   

                                                             name:es  \
element id                                                             
node    191031796  Punto kilométrico cero de las carreteras franc...   
        218117881                                                NaN   
        218119662                                                NaN   
        218145208                                                NaN   
        243461578                                                NaN   

                                           name:fr  ... image:0 max_age  \
element id                                          ...                   
node    191031796  Point zéro des Routes de France  ...     NaN     NaN   
        218117881                              NaN  ...     NaN     NaN   
        218119662                              NaN  ...     NaN     NaN   
        218145208                              NaN  ...     NaN     NaN   
        243461578                              NaN  ...     NaN     NaN   

                  shelter_type orientation parking_space:length  \
element id                                                        
node    191031796          NaN         NaN                  NaN   
        218117881          NaN         NaN                  NaN   
        218119662          NaN         NaN                  NaN   
        218145208          NaN         NaN                  NaN   
        243461578          NaN         NaN                  NaN   

                  parking_space:width service preschool name:br place  
element id                                                             
node    191031796                 NaN     NaN       NaN     NaN   NaN  
        218117881                 NaN     NaN       NaN     NaN   NaN  
        218119662                 NaN     NaN       NaN     NaN   NaN  
        218145208                 NaN     NaN       NaN     NaN   NaN  
        243461578                 NaN     NaN       NaN     NaN   NaN  

[5 rows x 527 columns]

In [58]:
import pandas as pd
from tqdm import tqdm  # barre de progression

# --- 1. Lire le fichier stops.txt---
stops = pd.read_csv("data/GTFS/stops.txt", sep=",")  # sépateur CSV standard
stops = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].dropna()

# Pour tester : prendre les 10 premières stations 
stops_sample = stops.head(10)

# --- 2. Définir les catégories de points d'intérêt ---
tags = {
    'amenity': True,   # cafés, écoles, hôpitaux...
    'shop': True,      # supermarchés, boulangeries...
    'leisure': True,   # parcs, piscines, stades...
    'tourism': True    # musées, monuments...
}

# --- 3. Fonction pour compter les POI autour d'une station ---
def count_poi_around_station(lat, lon, radius):
    try:
        gdf = ox.features_from_point((lat, lon), dist=radius, tags=tags)
        counts = {
            'n_amenities': gdf['amenity'].notna().sum(),
            'n_shops': gdf['shop'].notna().sum(),
            'n_leisure': gdf['leisure'].notna().sum(),
            'n_tourism': gdf['tourism'].notna().sum()
        }
        return counts
    except Exception as e:
        print(f"Erreur pour la station ({lat}, {lon}) : {e}")
        return {'n_amenities': 0, 'n_shops': 0, 'n_leisure': 0, 'n_tourism': 0}

# --- 4. Boucle principale ---
results = []

for _, row in tqdm(stops_sample.iterrows(), total=len(stops_sample)):
    lat, lon = row['stop_lat'], row['stop_lon']

    # POI dans un rayon de 400m
    c400 = count_poi_around_station(lat, lon, 400)
    # POI dans un rayon de 800m
    c800 = count_poi_around_station(lat, lon, 800)

    results.append({
        'stop_id': row['stop_id'],
        'stop_name': row['stop_name'],
        **{f'{k}_400m': v for k, v in c400.items()},
        **{f'{k}_800m': v for k, v in c800.items()}
    })

# --- 5. Résultats propres ---
poi_counts = pd.DataFrame(results)
print(poi_counts.head())

# --- 6. Sauvegarde ---
import os

# Crée le dossier si nécessaire
os.makedirs("Processed", exist_ok=True)
# Ensuite, on peut sauvegarder le CSV
poi_counts.to_csv("Processed/poi_counts_by_station.csv", sep=";", index=False)
print("Fichier enregistré : Processed/poi_counts_by_station.csv")

100%|██████████| 10/10 [00:04<00:00,  2.15it/s]

                          stop_id            stop_name  n_amenities_400m  \
0  IDFM:monomodalStopPlace:473875               Nation               548   
1   IDFM:monomodalStopPlace:53783     Chatou - Croissy               148   
2   IDFM:monomodalStopPlace:53784  Le Vésinet - Centre               121   
3  IDFM:monomodalStopPlace:473890    Denfert-Rochereau               569   
4  IDFM:monomodalStopPlace:473843   Cité Universitaire               619   

   n_shops_400m  n_leisure_400m  n_tourism_400m  n_amenities_800m  \
0           284              76              29              1911   
1            25              23               3               419   
2            79              11               2               207   
3           265              30              34              2049   
4            33              38              35              2160   

   n_shops_800m  n_leisure_800m  n_tourism_800m  
0           785             219              85  
1            66             

In [59]:
# Lire le CSV généré
poi_counts = pd.read_csv("Processed/poi_counts_by_station.csv", sep=";")
# Afficher les 10 premières lignes
poi_counts.head(10)   

,stop_id,stop_name,n_amenities_400m,n_shops_400m,n_leisure_400m,n_tourism_400m,n_amenities_800m,n_shops_800m,n_leisure_800m,n_tourism_800m
0,IDFM:monomodalStopPlace:473875,Nation,548,284,76,29,1911,785,219,85
1,IDFM:monomodalStopPlace:53783,Chatou - Croissy,148,25,23,3,419,66,90,16
2,IDFM:monomodalStopPlace:53784,Le Vésinet - Centre,121,79,11,2,207,95,86,2
3,IDFM:monomodalStopPlace:473890,Denfert-Rochereau,569,265,30,34,2049,798,121,154
4,IDFM:monomodalStopPlace:473843,Cité Universitaire,619,33,38,35,2160,289,184,107
5,IDFM:monomodalStopPlace:58267,Les Boullereaux Champigny,172,31,24,10,286,51,82,16
6,IDFM:monomodalStopPlace:58288,Les Yvris Noisy-le-Grand,229,1,5,13,243,3,20,17
7,IDFM:monomodalStopPlace:58270,Champigny,94,26,21,5,315,78,115,10
8,IDFM:monomodalStopPlace:46689,Viroflay Rive Droite,246,26,21,43,779,91,84,107
9,IDFM:monomodalStopPlace:46647,Achères Ville,136,4,12,16,188,7,20,17


Attention : Ne pas tester directement sur toutes les stations tout de suite :
→ OSM risque de bloquer les requêtes après quelques dizaines d’appels.
→ On pourra ensuite écrire un script qui met une pause automatique entre les requêtes ou qui utilise un cache local.

Attention, le code suivant tourne pendant des heures, ne pas le lancer !!

In [65]:
# ==============================
# Script complet POI + carte
# ==============================

import pandas as pd
import os
import time
from tqdm import tqdm
import folium
import osmnx as ox

# --- 1. Créer le dossier Processed si nécessaire ---
os.makedirs("Processed", exist_ok=True)

# --- 2. Lire stops.txt ---
stops = pd.read_csv("data/GTFS/stops.txt", sep=",")
stops.columns = stops.columns.str.strip()  # supprime espaces invisibles
stops = stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']].dropna()
print(f"Nombre de stations : {len(stops)}")

# --- 3. Définir les catégories de POI ---
tags = {
    'amenity': True,
    'shop': True,
    'leisure': True,
    'tourism': True
}

# --- 4. Fonction pour compter les POI autour d'une station ---
def count_poi_around_station(lat, lon, radius):
    try:
        gdf = ox.features_from_point((lat, lon), dist=radius, tags=tags)
        counts = {
            'n_amenities': gdf['amenity'].notna().sum(),
            'n_shops': gdf['shop'].notna().sum(),
            'n_leisure': gdf['leisure'].notna().sum(),
            'n_tourism': gdf['tourism'].notna().sum()
        }
        return counts
    except Exception as e:
        print(f"Erreur pour la station ({lat}, {lon}) : {e}")
        return {'n_amenities': 0, 'n_shops': 0, 'n_leisure': 0, 'n_tourism': 0}

# --- 5. Boucle principale pour toutes les stations ---
results = []

for i, row in tqdm(stops.iterrows(), total=len(stops)):
    lat, lon = row['stop_lat'], row['stop_lon']

    # POI dans un rayon de 400m et 800m
    c400 = count_poi_around_station(lat, lon, 400)
    c800 = count_poi_around_station(lat, lon, 800)

    results.append({
        'stop_id': row['stop_id'],
        'stop_name': row['stop_name'],
        'stop_lat': lat,
        'stop_lon': lon,
        **{f'{k}_400m': v for k, v in c400.items()},
        **{f'{k}_800m': v for k, v in c800.items()}
    })

    # Pause 1 seconde pour ne pas surcharger l'API Overpass
    time.sleep(1)

    # Sauvegarde intermédiaire toutes les 50 stations
    if i % 50 == 0 and i > 0:
        temp_df = pd.DataFrame(results)
        temp_df.to_csv("Processed/poi_counts_by_station_temp.csv", sep=";", index=False)

# --- 6. Sauvegarde finale ---
poi_counts = pd.DataFrame(results)
poi_counts.to_csv("Processed/poi_counts_by_station.csv", sep=";", index=False)
print("Fichier final sauvegardé : Processed/poi_counts_by_station.csv")

# --- 7. Carte Folium interactive ---
m = folium.Map(location=[48.8566, 2.3522], zoom_start=12)

for _, row in poi_counts.iterrows():
    folium.Marker(
        location=[row['stop_lat'], row['stop_lon']],
        popup=f"{row['stop_name']}\nAmenities 400m: {row['n_amenities_400m']}, Shops 400m: {row['n_shops_400m']}",
        icon=folium.Icon(color='blue', icon='train', prefix='fa')
    ).add_to(m)
    folium.Circle(location=[row['stop_lat'], row['stop_lon']], radius=400, color='green', fill=False).add_to(m)
    folium.Circle(location=[row['stop_lat'], row['stop_lon']], radius=800, color='red', fill=False).add_to(m)

# Sauvegarde la carte pour l’ouvrir dans le navigateur
m.save("Processed/poi_map.html")
print("Carte interactive sauvegardée : Processed/poi_map.html")


Nombre de stations : 53535


  0%|          | 20/53535 [10:22<774:37:56, 52.11s/it] 

Erreur pour la station (48.802662371849536, 1.8158352700699212) : 'leisure'
Erreur pour la station (48.802662371849536, 1.8158352700699212) : 'leisure'


  0%|          | 33/53535 [21:14<628:13:01, 42.27s/it] 

Erreur pour la station (48.51378614127599, 3.250114078411789) : 'leisure'


...

Erreur pour la station (49.00813565335945, 2.6802537675802918) : 'shop'
Erreur pour la station (49.00813565335945, 2.6802537675802918) : 'leisure'


  0%|          | 74/53535 [1:02:30<401:03:12, 27.01s/it]

Erreur pour la station (48.51507609100275, 2.5994097381795305) : 'shop'


  0%|          | 79/53535 [1:08:06<768:02:17, 51.72s/it] 


KeyboardInterrupt: 

In [66]:
print ("test2")

test2
